In [ ]:
logpath = r'./logs/logs18_nGAP'

import numpy as np
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
# import helpers # to get transpose softmax function
import torch.optim as optim
# from capsule_network_mm.dataset_Bup_mm import Dataset_mm
# from capsule_network_mm.read_pathfile_mm import read_path_label

from helpers_mm import softmax_mm
from dataset_Bup_mm_40 import Dataset_mm
from read_pathfile_mm import read_path_label
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import DRSNCS_nGAP as DRSNCS


TRAIN_ON_GPU= True

root_path = r'./data/dam_settle_data_sec_first_369_smote'

classes=['无破坏','轻度破坏','中度破坏','重度破坏']

batch_size = 15 # P100跑满内存只能设置15
learning_rate = 1e-3
is_lr_reduce = True
gamma = 0.8 #learning reduce factor

is_resize = False
n_epochs = 100
img_w = img_h = 112
img_c = 3

# 加载数据
root = root_path
train_images_path, train_images_label, test_images_path, test_images_label = read_path_label(root)

# dataset
train_dataset = Dataset_mm(train_images_path, train_images_label,outsize=img_w)
test_dataset = Dataset_mm(test_images_path, test_images_label,outsize=img_w)

train_data_size = len(train_images_label)
test_data_size = len(test_images_label)
print("训练集的长度为: {}".format(train_data_size))
print("测试集的长度为: {}".format(test_data_size))

# dataloader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size)

train_size = train_data_size
test_size = test_data_size

'''
定义网络
'''
# drsnet18
model = DRSNCS.rsnet18()
# drsnet34
# model = DRSNCS.rsnet34()


# # model summary
# from torchsummary import summary
# if torch.cuda.is_available() & TRAIN_ON_GPU:
#   summary(model.cuda(), input_size=[(3, img_w, img_h)], batch_size=batch_size, device="cuda")
# else:
#   summary(model, input_size=[(3, img_w, img_h)], batch_size=batch_size, device="cpu")



# fit parameter
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
if is_lr_reduce:
    #指数衰减调整学习率 ExponentialLR
    scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)


# gpu training
# 定义模型
device = torch.device("cuda:0" if torch.cuda.is_available()&TRAIN_ON_GPU else "cpu")
model.to(device)  # 移动模型到cuda
loss_fun = loss_fun.to(device)

# tensorboard
from  torch.utils.tensorboard.writer import SummaryWriter
writer = SummaryWriter(logpath)
init_img = torch.zeros((batch_size,img_c,img_w,img_h),device=device)
# writer.add_graph(model, init_img)

# 设置训练网络的一些参数
# #记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = n_epochs

best_acc = 0;

'''训练模型'''

for i in range(epoch):
    print("-------第{}轮训练开始-------".format(i+1))

    # 训练步骤开始
    model.train()
    total_train_acc = 0
    for data in train_loader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fun(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
        

        total_train_step = total_train_step + 1
        
#        if total_train_step % 100 == 0:
#            print("训练次数:{}，Loss:{}".format(total_train_step, loss.item()))
            

    # 测试步骤开始
    
    # 训练集精度
    model.eval()
    total_train_loss = 0
    total_train_acc = 0
    with torch.no_grad():
        for data in train_loader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fun(outputs, targets)
            total_train_loss = total_train_loss + loss.item()

            total_train_acc += (outputs.argmax(1) == targets).sum()
    print("整体训练集上的Loss: {}".format(total_train_loss/len(train_loader)))
    print("整体训练集上的正确率:{}".format(total_train_acc / train_size))
    
    # 测试集精度
    total_test_loss = 0
    total_test_acc = 0    
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    
    with torch.no_grad():
        for data in test_loader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fun(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            
            # 计算总准确率
            total_test_acc += (outputs.argmax(1) == targets).sum()
            
            #计算各类准确率
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(targets, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))
            
    print("整体测试集上的Loss: {}".format(total_test_loss/len(test_loader)))
    print("整体测试集上的正确率:{}".format(total_test_acc / test_size))
    # tensorboard
    writer.add_scalars("loss",
                      {"train_loss": total_train_loss/len(train_loader),
                      "test_loss": loss.item()}, total_test_step)
    writer.add_scalars("acc",
                      {"train_acc": total_train_acc/ train_size,
                      "test_acc": total_test_acc / test_size} , total_test_step)

    # tensorboard
    writer.add_scalars("classes_acc",
                      {"total_acc": total_test_acc / test_size,
                      "classes_acc_1": float(correct_pred[classes[0]]) / total_pred[classes[0]],
                      "classes_acc_2": float(correct_pred[classes[1]]) / total_pred[classes[0]],
                      "classes_acc_3": float(correct_pred[classes[2]]) / total_pred[classes[0]],
                      "classes_acc_4": float(correct_pred[classes[3]]) / total_pred[classes[0]]},total_test_step)
    
    #learning rate 
    lr_step = optimizer.state_dict()['param_groups'][0]['lr']
    print("本轮学习率：{}".format(lr_step))
    writer.add_scalar("learning_rate", lr_step, total_test_step)
    
    if is_lr_reduce:
        scheduler.step()
    total_test_step += 1
    
    #保存最佳模型
    if best_acc< total_test_acc / test_size :
        best_acc = total_test_acc / test_size
        torch.save(model.state_dict(), 'model34.pth')

writer.close()



2112 images were found in the dataset.
1692 images for training.
420 images for validation.
训练集的长度为: 1692
测试集的长度为: 420
-------第1轮训练开始-------
整体训练集上的Loss: 0.5539109942923605
整体训练集上的正确率:0.7257683277130127
Accuracy for class 无破坏   is: 100.0 %
Accuracy for class 轻度破坏  is: 42.9 %
Accuracy for class 中度破坏  is: 77.1 %
Accuracy for class 重度破坏  is: 78.1 %
整体测试集上的Loss: 0.5563494745375854
整体测试集上的正确率:0.7452381253242493
本轮学习率：0.001
-------第2轮训练开始-------
整体训练集上的Loss: 0.45317864292754534
整体训练集上的正确率:0.804964542388916
Accuracy for class 无破坏   is: 98.1 %
Accuracy for class 轻度破坏  is: 77.1 %
Accuracy for class 中度破坏  is: 78.1 %
Accuracy for class 重度破坏  is: 75.2 %
整体测试集上的Loss: 0.451285124257473
整体测试集上的正确率:0.8214285969734192
本轮学习率：0.0008
-------第3轮训练开始-------
整体训练集上的Loss: 0.267134956751777
整体训练集上的正确率:0.9054373502731323
Accuracy for class 无破坏   is: 96.2 %
Accuracy for class 轻度破坏  is: 81.9 %
Accuracy for class 中度破坏  is: 89.5 %
Accuracy for class 重度破坏  is: 93.3 %
整体测试集上的Loss: 0.296476559979575
整体测试集上的正确率:0.90238

整体训练集上的Loss: 0.03670390587984131
整体训练集上的正确率:0.9881796836853027
Accuracy for class 无破坏   is: 99.0 %
Accuracy for class 轻度破坏  is: 85.7 %
Accuracy for class 中度破坏  is: 84.8 %
Accuracy for class 重度破坏  is: 92.4 %
整体测试集上的Loss: 0.3067035934571842
整体测试集上的正确率:0.9047619104385376
本轮学习率：3.022314549036578e-06
-------第28轮训练开始-------
整体训练集上的Loss: 0.038383931372792716
整体训练集上的正确率:0.9881796836853027
Accuracy for class 无破坏   is: 99.0 %
Accuracy for class 轻度破坏  is: 86.7 %
Accuracy for class 中度破坏  is: 84.8 %
Accuracy for class 重度破坏  is: 91.4 %
整体测试集上的Loss: 0.29960031749214977
整体测试集上的正确率:0.9047619104385376
本轮学习率：2.4178516392292624e-06
-------第29轮训练开始-------
整体训练集上的Loss: 0.036422674618331735
整体训练集上的正确率:0.9899527430534363
Accuracy for class 无破坏   is: 99.0 %
Accuracy for class 轻度破坏  is: 87.6 %
Accuracy for class 中度破坏  is: 84.8 %
Accuracy for class 重度破坏  is: 92.4 %
整体测试集上的Loss: 0.30468215884840383
整体测试集上的正确率:0.9095238447189331
本轮学习率：1.93428131138341e-06
-------第30轮训练开始-------
整体训练集上的Loss: 0.03806383765624028
整体训

In [ ]:
out = model.conv1(imgs)
out = model.conv2_x(out)
out = model.conv3_x(out)
out = model.conv4_x(out)
out = model.conv5_x(out)
out = model.flatten(out)
out.shape


In [ ]:
25088/(7*7*512)